In [ ]:
!pip install selenium
!pip install webdriver_manager
!pip install aiohttp
!pip install nest_asyncio
!pip install azure-storage-blob

# Scraping

In [ ]:
import logging
import aiohttp
from bs4 import BeautifulSoup
import asyncio
import nest_asyncio
import random
import requests
import pandas as pd
from tqdm import tqdm

# Apply the nest_asyncio fix for running nested event loops
nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)


# --- Function to scrape product links ---
async def scrape_product_links(param: dict) -> list:
    first_page_url = param.get('first_page_url')
    base_url = param.get('base_url')

    product_links = []

    async with aiohttp.ClientSession() as session:
        # First page request
        async with session.get(first_page_url) as response:
            text = await response.text()
            soup = BeautifulSoup(text, 'html.parser')
            products = soup.find_all('div', class_='sc-19767e73-0 bwele')

            for product in products:
                link = product.find('a', href=True)['href']
                full_link = f"https://www.noon.com{link}"
                product_links.append(full_link)

        # Pagination loop for subsequent pages
        page = 2
        while True:
            url = base_url.format(page)
            async with session.get(url) as response:
                text = await response.text()
                soup = BeautifulSoup(text, 'html.parser')
                products = soup.find_all('div', class_='sc-19767e73-0 bwele')

                if not products:
                    break

                for product in products:
                    link = product.find('a', href=True)['href']
                    full_link = f"https://www.noon.com{link}"
                    product_links.append(full_link)

            page += 1

    return product_links  # Returning the list of product links


# --- Helper function for retries ---
async def fetch_with_retries(session, url, retries=3):
    for attempt in range(retries):
        try:
            async with session.get(url) as response:
                await asyncio.sleep(random.uniform(2, 7))  # To avoid being blocked by the server
                return await response.text()
        except Exception as e:
            logging.error(f"Error on attempt {attempt + 1} for {url}: {str(e)}")
            if attempt == retries - 1:
                raise  # Raise the exception if this was the last attempt


# --- Function to scrape product details ---
async def scrape_product_details(url: str) -> dict:
    async with aiohttp.ClientSession() as session:
        try:
            text = await fetch_with_retries(session, url)  # Retry logic added
            soup = BeautifulSoup(text, 'html.parser')

            # Extract the title
            title = soup.find('h1', class_='sc-a748f04d-17 bsHsUV').text.strip() if soup.find(
                'h1', class_='sc-a748f04d-17 bsHsUV') else 'N/A'

            # Extract the price
            price = soup.find('div', class_='priceNow').text.strip() if soup.find('div',
                                                                                class_='priceNow') else 'N/A'

            # Extract brand
            brand = soup.find('div', class_='sc-a748f04d-16 jLPVLQ').text.strip() if soup.find(
                'div', class_='sc-a748f04d-16 jLPVLQ') else 'N/A'

            # Extract rating
            rating = soup.find('div', class_='sc-9cb63f72-2 dGLdNc').text if soup.find(
                'div', class_='sc-9cb63f72-2 dGLdNc') else 'N/A'

            # Extract product image
            img_tags = soup.find_all('img')
            product_image = [
                img.get('src').split('?')[0] for img in img_tags
                if img.get('src') and img.get('src').startswith('https://f.nooncdn.com/') and img.get(
                    'src').endswith('width=240')
            ]

            # Extract the specifications table
            specifications = {}
            specs_table = soup.find('table').find('tbody').find_all('tr') if soup.find('table') else []

            for row in specs_table:
                header = row.find_all('td')[0].text.strip()
                value = row.find_all('td')[1].text.strip()
                specifications[header] = value

            # Add title, price, and other details to the specifications
            specifications.update({
                'Title': title,
                'Price': price,
                'Brand': brand,
                'Rating': rating,
                'Product Image': product_image,
                'Product Link': url
            })

            # Filter out products that have too many "N/A" fields
            if all(value == 'N/A' for value in [title, price, brand]):
                logging.warning(f"Incomplete data for product: {url}, skipping.")
                return None  # Skip this product if critical fields are missing

            return specifications

        except Exception as e:
            logging.error(f"Error fetching details from {url}: {str(e)}")
            return None  # Return None if there's an error


# --- Function to scrape multiple product details ---
async def scrape_multiple_products(urls: list) -> list:
    tasks = [scrape_product_details(url) for url in urls]  # Create a list of tasks for each URL
    results = await asyncio.gather(*tasks)  # Run them concurrently

    # Filter out any None results (products that were skipped)
    valid_results = [result for result in results if result]
    return valid_results


# --- Main integrated function ---
async def main(param: dict):
    # Step 1: Scrape product links
    product_links = await scrape_product_links(param)

    # Step 2: Scrape details for each product link
    product_details = await scrape_multiple_products(product_links)

    return product_details


# Function to run the entire scraping process
def main_function(req: dict):
    return asyncio.run(main(req))


# --- Example usage ---
if __name__ == "__main__":
    param = {
        'first_page_url': 'https://www.noon.com/egypt-en/electronics-and-mobiles/mobiles-and-accessories/mobiles-20905/eg-all-mobiles/?fisCarousel=true&fis_fbn=1&limit=50&sortby=popularity&sortdir=desc',  # Replace with actual first page URL
        'base_url': 'https://www.noon.com/egypt-en/electronics-and-mobiles/mobiles-and-accessories/mobiles-20905/eg-all-mobiles/?fisCarousel=true&fis_fbn=1&limit=50&page={}&sortby=popularity&sortdir=desc'  # Replace with actual pagination URL
    }

    # Run the scraper
    product_data = main_function(param)

    # Process the scraped data (e.g., store it in a file, analyze it, etc.)
    if product_data:
        df = pd.DataFrame(product_data)
        print(df.head())  # Display the first few rows of the DataFrame
        # ... (Add code for storing data or further analysis) ...

  Secondary Camera Resolution Charging Type SIM Count Secondary Camera  \
0                        8 MP        Type-C  Dual SIM      8 - 11.9 MP   
1                        8 MP        Type-C  Dual SIM      8 - 11.9 MP   
2                       16 MP        Type-C  Dual SIM     16 - 31.9 MP   
3                       13 MP        Type-C  Dual SIM     12 - 15.9 MP   
4                        2 MP        Type-C  Dual SIM      Upto 4.9 MP   

  RAM Size Battery Size Internal Memory              Version Screen Size  \
0     4 GB     5000 mAh          256 GB  Middle East Version      6.6 in   
1     4 GB     6000 mAh          128 GB  Middle East Version     6.75 in   
2    12 GB     5800 mAh          256 GB  Middle East Version     6.78 in   
3     8 GB     5000 mAh          256 GB  Middle East Version      6.5 in   
4     4 GB     5000 mAh          128 GB  Middle East Version      6.7 in   

   SIM Type  ... Refresh Rates Display Resolution Type Glass Technology  \
0  Nano SIM  ...       

In [ ]:
from datetime import datetime
import pandas as pd # Import the pandas library

# Get the current date and time
now = datetime.now()

# dd/mm/YY H:M:S
today = now.strftime("%d/%m/%Y:%H:%M")
today=pd.to_datetime(today, format="%d/%m/%Y:%H:%M") # Explicitly specify the format

In [ ]:
df=pd.DataFrame(product_data)
df.to_csv(f'noon_raw{today}.csv', index=False)
df.head()

,Secondary Camera Resolution,Charging Type,SIM Count,Secondary Camera,RAM Size,Battery Size,Internal Memory,Version,Screen Size,SIM Type,...,Refresh Rates,Display Resolution Type,Glass Technology,Video Recording Resolution Type,Finish,Camera Type,Battery Type,Pixels Per Inch,Network,Wireless
0,8 MP,Type-C,Dual SIM,8 - 11.9 MP,4 GB,5000 mAh,256 GB,Middle East Version,6.6 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8 MP,Type-C,Dual SIM,8 - 11.9 MP,4 GB,6000 mAh,128 GB,Middle East Version,6.75 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16 MP,Type-C,Dual SIM,16 - 31.9 MP,12 GB,5800 mAh,256 GB,Middle East Version,6.78 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13 MP,Type-C,Dual SIM,12 - 15.9 MP,8 GB,5000 mAh,256 GB,Middle East Version,6.5 in,Nano SIM,...,90Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 MP,Type-C,Dual SIM,Upto 4.9 MP,4 GB,5000 mAh,128 GB,Middle East Version,6.7 in,Nano SIM,...,90Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 57 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Secondary Camera Resolution      147 non-null    object
 1   Charging Type                    176 non-null    object
 2   SIM Count                        220 non-null    object
 3   Secondary Camera                 147 non-null    object
 4   RAM Size                         222 non-null    object
 5   Battery Size                     222 non-null    object
 6   Internal Memory                  222 non-null    object
 7   Version                          221 non-null    object
 8   Screen Size                      222 non-null    object
 9   SIM Type                         219 non-null    object
 10  Colour Name                      220 non-null    object
 11  Voice Calling Capability         96 non-null     object
 12  Processor Number                 113

# Saving the output of the scraping code to the datalake

In [ ]:
from azure.storage.blob import BlobServiceClient

# Your Azure connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=noon;AccountKey=d2CI4UUSt0o5owVHuKgR/baEGSWZ2vMWIAD3p38DCsScpHUALrqIheG4EIyLbM809DGlwrTXNU0B+AStzkSOnw==;EndpointSuffix=core.windows.net"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Local file path
local_file_path = f"noon_raw{today}.csv"

# Container and blob details
container_name = "proj"
blob_name = f"Raw/noon_raw{today}.csv"  # File in 'Transformed' folder

# Create a BlobClient
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data)

print(f"File {local_file_path} uploaded to container '{container_name}' under '{blob_name}'")


File /content/noon_raw2024-10-20 03:33:00.csv uploaded to container 'proj' under 'Raw/noon_raw2024-10-20 03:33:00.csv'


# Transformation

####  Reading the data from the datalake

In [ ]:
# to avoid consuming the stream before reading
from io import BytesIO
download_stream = blob_client.download_blob().readall()
data_stream = BytesIO(download_stream)

# Read the data into a Pandas DataFrame
df = pd.read_csv(data_stream)

# Now you have the data in a DataFrame called 'df'
df.head()

,Secondary Camera Resolution,Charging Type,SIM Count,Secondary Camera,RAM Size,Battery Size,Internal Memory,Version,Screen Size,SIM Type,...,Refresh Rates,Display Resolution Type,Glass Technology,Video Recording Resolution Type,Finish,Camera Type,Battery Type,Pixels Per Inch,Network,Wireless
0,8 MP,Type-C,Dual SIM,8 - 11.9 MP,4 GB,5000 mAh,256 GB,Middle East Version,6.6 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8 MP,Type-C,Dual SIM,8 - 11.9 MP,4 GB,6000 mAh,128 GB,Middle East Version,6.75 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16 MP,Type-C,Dual SIM,16 - 31.9 MP,12 GB,5800 mAh,256 GB,Middle East Version,6.78 in,Nano SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13 MP,Type-C,Dual SIM,12 - 15.9 MP,8 GB,5000 mAh,256 GB,Middle East Version,6.5 in,Nano SIM,...,90Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 MP,Type-C,Dual SIM,Upto 4.9 MP,4 GB,5000 mAh,128 GB,Middle East Version,6.7 in,Nano SIM,...,90Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# selecting the necessary columns

selected_columns=['RAM Size',
 'Battery Size',
 'Internal Memory',
 'Screen Size',
 'Colour Name',
 'Title',
 'Price',
 'Brand',
 'Rating',
 'Product Image',
 'Product Link',
 'Version',
 'Model Name',
 'Primary Camera (MP)',
 'Secondary Camera Resolution',
 'Refresh Rates',
 'Operating System',
 'Operating System Version',
 'Processor Speed',
 'Network Type',
 'Model Year',
 'Display Resolution']

df=df[selected_columns]
df.rename(columns={'Operating System':'os','Operating System Version':'os_version','RAM Size':'ram',
                         'Internal Memory':'storage'}, inplace=True)

#drop nulls in model name
df.dropna(subset=['Model Name'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Convert all values to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-220-78084be7c99b>:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ram                          212 non-null    object 
 1   Battery Size                 212 non-null    object 
 2   storage                      212 non-null    object 
 3   Screen Size                  212 non-null    object 
 4   Colour Name                  212 non-null    object 
 5   Title                        212 non-null    object 
 6   Price                        212 non-null    object 
 7   Brand                        212 non-null    object 
 8   Rating                       208 non-null    float64
 9   Product Image                212 non-null    object 
 10  Product Link                 212 non-null    object 
 11  Version                      211 non-null    object 
 12  Model Name                   212 non-null    object 
 13  Primary Camera (MP) 

In [ ]:
import re
spec=[]
def parse_camera_spec(camera_spec):
    # Check if camera_spec is a valid string
    if isinstance(camera_spec, str):
        # Remove non-numeric characters except for plus signs and split by plus
        numbers = re.findall(r'\d+', camera_spec)

        # Convert the list of strings into integers
        return [int(num) for num in numbers]
    else:
        # Handle missing or invalid values
        return []

# Iterate over the column to apply the function to each row
for camera_spec in df['Primary Camera (MP)']:
  output = parse_camera_spec(camera_spec)
  spec.append(sorted(output,reverse=True))

df['Primary Camera (MP)']=spec

def sum_list_elements(lst):
  if isinstance(lst, list):
    return sum(lst)
  else:
    return lst  # Handle cases where it's not a list

df['Primary Camera (MP)'] = df['Primary Camera (MP)'].apply(sum_list_elements)

In [ ]:
sam_df=df[df['Model Name'].str.contains('sm-')]
sam_df['Model Name']
sam_phones=[]
specific_elements=['galaxy','dual','sim']
for i in sam_df['Title'].str.split():
  sam_phones.append(' '.join([x for x in i[0:4] if x not in specific_elements]))

# Create a dictionary to map the indices to sam_phones values
sam_phones_dict = {i: val for i, val in enumerate(sam_phones)}

# Use map to replace the values in the Model Name column
df.loc[df['Model Name'].str.contains('sm-'), 'Model Name'] = df.groupby(df['Model Name'].str.contains('sm-')).ngroup().map(sam_phones_dict)
unused=['oppo','galaxy','gray','titanium','samsung','black','blue','nokia','realme','iphone','4g','5g','oneplus','mobile phone']
for i in unused:
  df['Model Name']=df['Model Name'].str.replace(i,'')

df['Model Name']=df['Model Name'].str.split(',').str[0].str.replace('+',' plus')



In [ ]:
# Function to clean and extract model name
def extract_model_name(model_name):
    # Use regular expression to capture the main model name (alphanumeric + spaces)
    # This pattern extracts only the first two alphanumeric words
    match = re.match(r'^([\w\d\s]+)\b', model_name)

    if match:
        # Return the extracted model name, strip any extra spaces
        return match.group(1).strip()
    else:
        return model_name  # Return original if no match

# Apply the function to the "Model Name" column
df['Model Name'] = df['Model Name'].apply(extract_model_name)

df['os_version']=df['os_version'].str.split().str[1]

In [ ]:
from datetime import datetime
# Assuming 'column_name' is the column you want to clean
df['Price'] = df['Price'].str.extract('(\d+\.?\d*)').astype(float).fillna(0).astype(int) # Changed to float first, then filled NaN with 0, then int

# Extract the digits and convert to float
df['ram'] = df['ram'].str.extract('(\d+\.?\d*)').astype(float).fillna(0).astype(int)

# Extract the digits and convert to float
df['Screen Size'] = df['Screen Size'].str.extract('(\d+\.?\d*)').astype(float).fillna(0)
df['storage'] = df['storage'].str.extract('(\d+\.?\d*)').astype(float).fillna(0).astype(int)
df['Refresh Rates'] = df['Refresh Rates'].str.extract('(\d+\.?\d*)').astype(float).fillna(0).astype(int)
df['Processor Speed'] = df['Processor Speed'].str.extract('(\d+\.?\d*)').astype(float).fillna(0)
df['Secondary Camera Resolution']=df['Secondary Camera Resolution'].str.split().str[0]


# Add the new column with the date value
df['date'] = datetime.now().date()

# Reorder the DataFrame to place the new column first
df = df[['date'] + [col for col in df.columns if col != 'date']]
df['site'] = 'noon'
df['category'] = 'mobile phones'

<ipython-input-225-b42beaa7301a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['site'] = 'noon'
<ipython-input-225-b42beaa7301a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = 'mobile phones'


In [ ]:
# Replace all zeroes with NaNs
import numpy as np
import re

df.replace(0, np.nan, inplace=True)
# Replace empty strings and spaces with NaNs
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# Convert `date_column` to datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')

# Convert `price_usd` to numeric
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Convert `ram_gb`, `storage`, `screen_size_in`, `refresh_rate_hz`, `cpu_speed_ghz` to numeric
df['ram'] = pd.to_numeric(df['ram'], errors='coerce')
df['storage'] = pd.to_numeric(df['storage'], errors='coerce')
df['Screen Size'] = pd.to_numeric(df['Screen Size'], errors='coerce')
df['Refresh Rates'] = pd.to_numeric(df['Refresh Rates'], errors='coerce')
df['Processor Speed'] = pd.to_numeric(df['Processor Speed'], errors='coerce')
df['Secondary Camera Resolution']=df['Secondary Camera Resolution'].astype(float)
# Clean and convert 'Battery Size' to numeric
df['Battery Size'] = df['Battery Size'].str.replace(r"[a-zA-Z\s]+", "", regex=True)  # Remove all letters and spaces
df['Battery Size'] = pd.to_numeric(df['Battery Size'], errors='coerce').fillna(0).astype(int)  # Convert to numeric, handle errors, fill NaN with 0
df['Model Year']=pd.to_numeric(df['Model Year'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         212 non-null    datetime64[ns]
 1   ram                          212 non-null    int64         
 2   Battery Size                 212 non-null    int64         
 3   storage                      212 non-null    int64         
 4   Screen Size                  212 non-null    float64       
 5   Colour Name                  212 non-null    object        
 6   Title                        212 non-null    object        
 7   Price                        212 non-null    int64         
 8   Brand                        212 non-null    object        
 9   Rating                       208 non-null    float64       
 10  Product Image                212 non-null    object        
 11  Product Link                 212 non-null    

<ipython-input-226-8da700f6b307>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(0, np.nan, inplace=True)
<ipython-input-226-8da700f6b307>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
<ipython-input-226-8da700f6b307>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], forma

In [ ]:
df.to_csv(f'noon_transformed{today}.csv', index=False)
df.head()

,date,ram,Battery Size,storage,Screen Size,Colour Name,Title,Price,Brand,Rating,...,Secondary Camera Resolution,Refresh Rates,os,os_version,Processor Speed,Network Type,Model Year,Display Resolution,site,category
0,2024-10-20,4,5000,256,6.60,starlish black,a70 dual sim starlish black 4gb ram 256gb 4g -...,4299,itel,4.2,...,8.0,NaN,NaN,NaN,NaN,4g,NaN,NaN,noon,mobile phones
1,2024-10-20,4,6000,128,6.75,midnight black,x7a dual sim midnight black 4gb ram 128gb 4g l...,4875,honor,4.3,...,8.0,NaN,android,12,2.3,4g,2023.0,720x1600,noon,mobile phones
2,2024-10-20,12,5800,256,6.78,midnight black,x9b dual sim 5g midnight black 12gb ram 256gb ...,13550,honor,4.4,...,16.0,NaN,android,NaN,2.0,5g,2023.0,NaN,noon,mobile phones
3,2024-10-20,8,5000,256,6.50,blue/black,galaxy a15 dual sim blue black 8gb ram 256gb 4...,8666,samsung,4.3,...,13.0,90.0,android,NaN,NaN,4g,2023.0,NaN,noon,mobile phones
4,2024-10-20,4,5000,128,6.70,silver,galaxy a05s dual sim silver 4gb ram 128gb 4g l...,5799,samsung,4.2,...,2.0,90.0,android,NaN,NaN,4g,NaN,1080x2400,noon,mobile phones


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         212 non-null    datetime64[ns]
 1   ram                          212 non-null    int64         
 2   Battery Size                 212 non-null    int64         
 3   storage                      212 non-null    int64         
 4   Screen Size                  212 non-null    float64       
 5   Colour Name                  212 non-null    object        
 6   Title                        212 non-null    object        
 7   Price                        212 non-null    int64         
 8   Brand                        212 non-null    object        
 9   Rating                       208 non-null    float64       
 10  Product Image                212 non-null    object        
 11  Product Link                 212 non-null    

# upload transformed data

In [ ]:
# blob details
blob_name = f"Transformed/noon_transformed{today}.csv"  # File in 'Transformed' folder

# Create a BlobClient
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data)

print(f"File {local_file_path} uploaded to container '{container_name}' under '{blob_name}'")


File /content/noon_raw2024-10-20 03:33:00.csv uploaded to container 'proj' under 'Transformed/noon_transformed2024-10-20 03:33:00.csv'


# Data Modeling

## Creating Dimensions and Fact Table

In [ ]:
# import pandas as pd
# import numpy as np
# import hashlib
# from azure.storage.blob import BlobServiceClient
# import os


# # Step : Create a function to generate a unique product_id using hashlib
# def generate_product_id(row):
#     # Combine columns to create a unique string
#     unique_string = (str(row['Model Name']) +
#                      str(row['Brand']) +
#                      str(row['Network Type']) +
#                      str(row['ram']) +
#                      str(row['storage']))
#     return hashlib.md5(unique_string.encode()).hexdigest()

# # Apply the function to generate product_id
# df['product_id'] = df.apply(generate_product_id, axis=1)

# # Step 3: Create unique dates and Dim_Date table
# unique_dates = df['date'].drop_duplicates().reset_index(drop=True)
# dim_date = pd.DataFrame({
#     'Date_ID': np.arange(len(unique_dates)),  # Create Date_ID
#     'Date': unique_dates,
#     'Day': unique_dates.dt.day,
#     'Month': unique_dates.dt.month,
#     'Year': unique_dates.dt.year
# })

# # Merge to add Date_ID to the original dataframe
# df = df.merge(dim_date[['Date_ID', 'Date']], left_on='date', right_on='Date', how='left', suffixes=('', '_dim'))

# # Step 4: Fact Table
# fact_table = pd.DataFrame({
#     'product_id': df['product_id'],  # Using the generated product_id
#     'site_id': 1,  # Placeholder; you should have a mapping for site_id
#     'date_id': df['Date_ID'],  # Using Date_ID from Dim_Date
#     'price_EGP': df['Price'],
#     'rating_avg': df['Rating']
# })


# # Create the 'Model' directory if it doesn't exist
# os.makedirs('Model', exist_ok=True)
# # Save Fact Table as Parquet
# fact_table.to_parquet('Model/fact_table.parquet')




# # Step 5: Dim_DeviceSpecifications Table
# dim_device_spec = df[['Battery Size','Screen Size', 'Colour Name',
#        'Title', 'Product Image', 'Product Link',
#        'Version', 'Primary Camera (MP)',
#        'Secondary Camera Resolution', 'Refresh Rates', 'os', 'os_version',
#        'Processor Speed', 'Model Year', 'Display Resolution',
#        'site', 'category']].copy()
# dim_device_spec['product_id'] = df['product_id']  # Use the generated product_id

# # Step 6: Create Dim_Map_ID Table
# dim_map_id = df[['product_id', 'storage', 'ram', 'Brand', 'Model Name', 'Network Type']].copy()

# # Save Dim_Map_ID as Parquet
# dim_map_id.to_parquet('Model/dim_map_id.parquet')

# # Save Dim_DeviceSpecifications as Parquet
# dim_device_spec.to_parquet('Model/dim_device_specifications.parquet')

# # Step 7: Dim_Site Table
# dim_site = df[['site']].drop_duplicates().reset_index(drop=True)
# dim_site['site_id'] = dim_site.index
# dim_site.columns = ['site_name', 'site_id']

# # Save Dim_Site as Parquet
# dim_site.to_parquet('Model/dim_site.parquet')

# # Step 9: Save Dim_Date as Parquet
# dim_date.to_parquet('Model/dim_date.parquet')

# # Step 10: Upload all Parquet files to Azure Blob Storage

# ## Loop through the parquet files in the Model directory and upload them
# model_dir = 'Model'
# for filename in os.listdir(model_dir):
#     local_file_path = os.path.join(model_dir, filename)

#     # Check if it's a file before uploading
#     if os.path.isfile(local_file_path):
#         blob_name = f'Model/{filename}'

#         blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

#         # Upload the file
#         with open(local_file_path, "rb") as data:
#             blob_client.upload_blob(data, overwrite=True)

#         print(f"File {filename} uploaded to container '{container_name}' under '{blob_name}'")



# Updating

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from azure.storage.blob import BlobServiceClient
import os
from io import BytesIO

# Function to download existing data from Azure Blob Storage
def download_blob_as_df(blob_service_client, container_name, blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    try:
        # Download blob to a DataFrame
        with open("temp.parquet", "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())
        return pd.read_parquet("temp.parquet")
    except:
        # Return an empty DataFrame if the blob doesn't exist
        return pd.DataFrame()

# Load the newly scraped dataset
df_new = df
# Convert the 'date' column to datetime format
df_new['date'] = pd.to_datetime(df_new['date'], errors='coerce')

# Step 1: Update Dim_Date
# Create unique dates and Dim_Date table for the new data
unique_dates = df_new['date'].drop_duplicates().reset_index(drop=True)
dim_date_new = pd.DataFrame({
    'Date_ID': np.arange(len(unique_dates)),
    'Date': unique_dates,
    'Day': unique_dates.dt.day,
    'Month': unique_dates.dt.month,
    'Year': unique_dates.dt.year
})

# Download existing Dim_Date from Azure Blob Storage
dim_date_existing = download_blob_as_df(blob_service_client, container_name, "Model/dim_date.parquet")

# Combine new dates with existing dates
if not dim_date_existing.empty:
    dim_date_updated = pd.concat([dim_date_existing, dim_date_new], ignore_index=True).drop_duplicates(subset=['Date'])
else:
    dim_date_updated = dim_date_new


# Reorder to keep the original Date_ID order
dim_date_updated = dim_date_updated.sort_values('Date_ID').reset_index(drop=True)


# Ensure the 'Model' directory exists
if not os.path.exists('Model'):
    os.makedirs('Model')

# Now save the updated Dim_Date table
dim_date_updated.to_parquet('Model/dim_date.parquet')

# Upload updated Dim_Date to Azure Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob="Model/dim_date.parquet")
with open("Model/dim_date.parquet", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print("Dim_Date updated and uploaded successfully.")
# Merge the new dataset (df_new) with the updated dim_date to get Date_ID in df_new
df_new = df_new.merge(dim_date_updated[['Date_ID', 'Date']], left_on='date', right_on='Date', how='left', suffixes=('', '_dim'))

# Function to generate product_id using hashlib
def generate_product_id(row):
    # Combine columns to create a unique string
    unique_string = (str(row['Model Name']) +
                     str(row['Brand']) +
                     str(row['Network Type']) +
                     str(row['ram']) +
                     str(row['storage']))
    return hashlib.md5(unique_string.encode()).hexdigest()


# Apply the function to generate product_id for new data
df_new['product_id'] = df_new.apply(generate_product_id, axis=1)

# Step 2: Update Dim_Site
dim_site_existing = download_blob_as_df(blob_service_client, container_name, "Model/dim_site.parquet")
dim_site_new = df_new[['site']].drop_duplicates().reset_index(drop=True)
dim_site_new['site_id'] = dim_site_new.index
dim_site_new.columns = ['site_name', 'site_id']

if not dim_site_existing.empty:
    dim_site_updated = pd.concat([dim_site_existing, dim_site_new], ignore_index=True).drop_duplicates(subset=['site_name'])
else:
    dim_site_updated = dim_site_new

dim_site_updated = dim_site_updated.sort_values('site_id').reset_index(drop=True)

# Save the updated Dim_Site table
dim_site_updated.to_parquet('Model/dim_site.parquet')

# Upload updated Dim_Site to Azure Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob="Model/dim_site.parquet")
with open("Model/dim_site.parquet", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)


# Step 2: Download existing fact_table from Azure
fact_table_existing = download_blob_as_df(blob_service_client, container_name, "Model/fact_table.parquet")

# Merge new and existing data for Fact Table
if not fact_table_existing.empty:
    fact_table_existing.set_index('product_id', inplace=True)
    fact_table_new = pd.DataFrame({
        'product_id': df_new['product_id'],
        'site_id': 2,  # Assuming site_id is constant or unchanged
        'date_id': df_new['Date_ID'],  # Use Date_ID generated above
        'price_EGP': df_new['Price'],
        'rating_avg': df_new['Rating']
    })
    fact_table_new.set_index('product_id', inplace=True)

    # Update existing records, and append new ones
    fact_table_updated = fact_table_existing.combine_first(fact_table_new)
else:
    fact_table_updated = pd.DataFrame({
        'product_id': df_new['product_id'],
        'site_id': 1,
        'date_id': df_new['Date_ID'],
        'price_EGP': df_new['Price'],
        'rating_avg': df_new['Rating']
    })

# Step 3: Save the updated Fact Table
fact_table_updated.reset_index(inplace=True)
fact_table_updated.to_parquet('Model/fact_table.parquet')

# Upload updated Fact Table to Azure Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob="Model/fact_table.parquet")
with open("Model/fact_table.parquet", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

# Step 4: Update Dim_DeviceSpecifications
dim_device_spec_existing = download_blob_as_df(blob_service_client, container_name, "Model/dim_device_specifications.parquet")
dim_device_spec_new = df_new[['product_id', 'Battery Size','Screen Size', 'Colour Name',  # Include 'product_id' here
        'Title', 'Product Image', 'Product Link',
        'Version', 'Primary Camera (MP)',
        'Secondary Camera Resolution', 'Refresh Rates', 'os', 'os_version',
        'Processor Speed', 'Model Year', 'Display Resolution',
        'site', 'category']]

if not dim_device_spec_existing.empty:
    dim_device_spec_existing.set_index('product_id', inplace=True)
    dim_device_spec_new.set_index('product_id', inplace=True)
    dim_device_spec_updated = dim_device_spec_existing.combine_first(dim_device_spec_new)
else:
    dim_device_spec_updated = dim_device_spec_new

# Save the updated Dim_DeviceSpecifications table
dim_device_spec_updated.reset_index(inplace=True)
dim_device_spec_updated.to_parquet('Model/dim_device_specifications.parquet')

# Upload updated Dim_DeviceSpecifications to Azure Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob="Model/dim_device_specifications.parquet")
with open("Model/dim_device_specifications.parquet", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

# Step 5: Update Dim_Map_ID
dim_map_id_existing = download_blob_as_df(blob_service_client, container_name, "Model/dim_map_id.parquet")
dim_map_id_new = df_new[['product_id', 'storage', 'ram', 'Brand', 'Model Name', 'Network Type']]

if not dim_map_id_existing.empty:
    dim_map_id_existing.set_index('product_id', inplace=True)
    dim_map_id_new.set_index('product_id', inplace=True)
    dim_map_id_combined = pd.concat([dim_map_id_existing, dim_map_id_new], ignore_index=False)

    # Drop duplicates based on 'product_id' to ensure unique entries
    dim_map_id_updated = dim_map_id_combined[~dim_map_id_combined.index.duplicated(keep='first')]
else:
    dim_map_id_updated = dim_map_id_new.drop_duplicates(subset=['product_id'])

# Save the updated Dim_Map_ID table
dim_map_id_updated.reset_index(inplace=True)
dim_map_id_updated.to_parquet('Model/dim_map_id.parquet')

# Upload updated Dim_Map_ID to Azure Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob="Model/dim_map_id.parquet")
with open("Model/dim_map_id.parquet", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)




print("Data processing and upload to Azure Blob Storage completed successfully!")


Data processing and upload to Azure Blob Storage completed successfully!
